# Image annotation and feature vector generation script

This notebook will serve as a prototype to create a script to annotate and create fecture vectors for the images of Terzani collection

## Installing the libraries

### Install `dotenv` to get the annotations of images

If you have python environment use

```shell
pip install --upgrade google-cloud-vision
```

If you have conda environment use

```shell
conda install -c conda-forge google-cloud-vision
```

### Install `dotenv` to get the environment variables

If you have python environment use

```shell
pip install python-dotenv
```

If you have conda environment use

```shell
conda install -c conda-forge python-dotenv
```

### Install `nltk` to process text

If you have python environment use

```shell
pip install nltk
```

If you have conda environment use

```shell
conda install -c anaconda nltk
```

### Install `pytorch` to get the feature vectors

This installation refers to the system, look [here](https://pytorch.org/get-started/locally/) for more details 

## Import the libraries

In [1]:
## Import the standard libraries
import os, io, pickle, random, json
## Import Vison API related libraries
from google.cloud import vision
from google.cloud.vision import types
## Import dotenv library to get environment variables
from dotenv import load_dotenv
# Import urllib to read images
import urllib.request as ur
# Import pymango to inset data into mangodb
import pymongo
from tqdm import tqdm
# Import nltk to process text
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
import nltk
import string
nltk.download('punkt')
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image 
import matplotlib.pyplot as plt
import requests
from io import BytesIO

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ravin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Setup the service account credentials to use the API

In [2]:
load_dotenv()

GOOGLE_APPLICATION_CREDENTIALS = os.getenv('GOOGLE_APPLICATION_CREDENTIALS')
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = GOOGLE_APPLICATION_CREDENTIALS

MANGO_CLIENT_URI = os.getenv('MONGO_URI')
os.environ['MANGO_CLIENT_URI'] = MANGO_CLIENT_URI

## Creation of Client to access the API

In [3]:
client = vision.ImageAnnotatorClient()

## Image selection

As this is a prototyping script we shall select 10 images randomly each from the color and monochrome photos (using the already created pickle files).

Generic class to store an image and its IIIF representation

In [4]:
class Terzani_Photo(object):
    def __init__(self, iiif, country):
        self.iiif = iiif
        self.photo = country
        
    def get_photo_link(self):
        return self.iiif["images"][0]["resource"]["@id"]

In [5]:
nu_of_images_per_type = 40

'''
# loading the color photos
color_photos = pickle.load(open("terzani_recto_iiif_color.pickle", "rb" ))
# randomly selecting 10 images
color_photos = random.sample(color_photos, nu_of_images_per_type)

# loading the monochrome photos
bw_photos = pickle.load(open("terzani_recto_iiif_color.pickle", "rb" ))
# randomly selecting 10 images
bw_photos = random.sample(bw_photos, nu_of_images_per_type)

all_photos = color_photos + bw_photos

'''

all_photos = pickle.load(open("terzani_recto_iiif.pickle", "rb"))
all_photos = random.sample(all_photos, nu_of_images_per_type)

## Function to clean text

In [6]:
def clean_text(text: str, lower: bool=True, rmv_punc: bool = True, stem: bool = True, norm: bool = True):
    """
    This function accepts a string and performs preprocessing steps on it. 
    
    :param text (str): The string or text on which the preprocessing has to be performed.
    :param lower (bool): Default=True, indicates if the text has to be made into lower case.
    :param rmv_punc (bool): Default=True, indicates if the punctuation should be removed in the text.
    :param stem (bool): Default=True, indicates if the stemming should be performed on the words in the text.
    :param norm (bool): Default=True, indicates if the words in the has to be normalised.
    :return cleaned_text (list): The modified text is returned as list after performing the indicated operations.
    """
    
    # split into words
    tokens = word_tokenize(text)
    if lower:
        # convert to lower case
        tokens = [w.lower() for w in tokens]
    if rmv_punc:
        # remove punctuation from each word
        table = str.maketrans('', '', string.punctuation)
        tokens = [w.translate(table) for w in tokens if w.translate(table) != '']
    if stem:
        # stemming of words
        porter = PorterStemmer()
        tokens = [porter.stem(word) for word in tokens]
    cleaned_text = sorted(list(set(tokens)),key=str.lower)
    return cleaned_text

## Function to get feature vector of an Image

In [7]:
# Loading pretrained Resnet model
model = models.resnet18(pretrained=True)
layer = model._modules.get('avgpool')
# Set model to evaluation mode as we plan to use pre trained model
model.eval()
scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

def transform_image(image):
    return Variable(normalize(to_tensor(scaler(image))).unsqueeze(0))

def get_vector(image_url):
    response = requests.get(image_url)
    img = Image.open(BytesIO(response.content))
    t_img = transform_image(img)
    #  The avgpool layer that we selected has an output size of 512
    my_embedding = torch.zeros(512)
    def copy_data(m, i, o):
        my_embedding.copy_(o.data.squeeze())
    h = layer.register_forward_hook(copy_data)
    model(t_img)
    h.remove()
    return my_embedding

## Calling the Vision API

The structure of tagged_images will be
```json
{'tag1': [label_1],
 'tag2': [label_2, label_4],
 'tag3': [label_3],
 'tag4': [label_3]
}
```
 
The structure of annotated_images will be
```json
{'label_1':
    {
    'iiif': IIIF ANNOTATION,
    'obj_boxes':
    {
    frozenset({'tag1'}): [[x1, y1, w1, h1]],
    frozenset({'tag2'}): [[x4, y4, w4, h4], [x2, y2, w2, h2], [x3, y3, w3, h3]],
    frozenset({'tag3', 'tag4'}): [[x5, y5, w5, h5], [x6, y6, w6, h6], [x7, y7, w7, h7]]
    },
    'landmark_info':
    {
    'landmark1_name':{"latitude":la1, "longitude":lt1},
    'landmark2_name':{"latitude":la2, "longitude":lt2}
    },
    'country': None
    },
'label_2':
    {
    'iiif': IIIF ANNOTATION,
    'obj_boxes':
    {
    frozenset({'tag4'}): [[x1, y1, w1, h1]],
    frozenset({'tag5'}): [[x4, y4, w4, h4],],
    frozenset({'tag6'}): [[x5, y5, w5, h5], [x6, y6, w6, h6]]
    },
    'landmark_info':
    {
    'landmark3_name':{"latitude":la3, "longitude":lt3},
    'landmark4_name':{"latitude":la4, "longitude":lt4}
    },
    'country': None
    }
}
```

The structure of image_vectors will be
```json
{'label_1': [<feature Array>],
 'label_2': [<feature Array>],
 'label_3': [<feature Array>],
 'label_4': [<feature Array>]
}
```

In [9]:
tagged_images = dict() # The keys would be the tags, entities and objects found in the annotation and the values would be the image labels.
annotated_images = dict() # The keys would be the image labels and the values will be the IIIF annotation, name of the country, lat,lon if there is geotag and object localization.
annotation_failed_images = dict() # We store information about images that are failed to be annotated by google api.

image_vecs = dict() # The keys would be the image labels and the values feature vectors generated using Resnet50
fvector_failed_images= list() # The list of images for which feature vectors were not created

for img in tqdm(all_photos):

    # if the image is already not present in the either annotated and failed dictionaries
    if img.iiif["label"] not in annotated_images and img.iiif["label"] not in annotation_failed_images and img.iiif["label"] not in image_vecs and img.iiif["label"] not in fvector_failed_images:

        # get the image label
        img_lbl = img.iiif["label"]
        img_country = img.country
                
        # reading the image
        image_data = ur.urlopen(img.get_photo_link()).read()
        image = types.Image(content=image_data)
        
        # call the goole vision api to get the annotations of various types
        response = client.annotate_image({
            'image': image,
            'features': [{'type': vision.enums.Feature.Type.LANDMARK_DETECTION}, 
                         {'type': vision.enums.Feature.Type.LOGO_DETECTION},
                         {'type': vision.enums.Feature.Type.LABEL_DETECTION},
                         {'type': vision.enums.Feature.Type.TEXT_DETECTION},
                         {'type': vision.enums.Feature.Type.OBJECT_LOCALIZATION},
                         {'type': vision.enums.Feature.Type.WEB_DETECTION}],})
        
        # check if there is any error returned by the api
        if response.error.code != 0:
            annotation_failed_images[img_lbl] = {}
            annotation_failed_images[img_lbl]["error"] = [response.error.code, response.error.message]
        else:
            # if the API call is successful
            annotated_images[img_lbl] = {}
        
            # store the iiif description
            annotated_images[img_lbl]["iiif"] = img.iiif

            # get the list of labels
            labels = list()
            for lbl in response.label_annotations:
                labels.extend(clean_text(lbl.description))
            labels = list(set(labels))
            
            # Add the label and image label to the dictionary to perform search
            for label in labels:
                if label not in tagged_images:
                    tagged_images[label] = []
                if img_lbl not in tagged_images[label]:
                    tagged_images[label].append(img_lbl)
            
            # get the list of web entities
            webent = list()
            for weben in response.web_detection.web_entities:
                webent.extend(clean_text(weben.description))
            webent = list(set(webent))
            
            # Add the web entity and image label to the dictionary to perform search
            for web_entity in labels:
                if web_entity not in tagged_images:
                    tagged_images[web_entity] = []
                if img_lbl not in tagged_images[web_entity]:
                    tagged_images[web_entity].append(img_lbl)
            

            obj_boxes = {} # this dictionary will store the information of annotations along with bounding boxes.
            # The key will the the name to identify the annotation and the value be a list of lists containing the top left x coordinate
            # top left y coordinate, width and height of for the bounding box.
            # It would be a list of list to store coordinates for different boxes for same tag

            # storing the landmarks
            landmark_info = dict() # this dictionary will store the information of landmarks which are name, latitude, longitude.
            for lndmk in response.landmark_annotations:
                print("landmark detected !")
                # if there are any landamrks identified, we store them in a seperate field,to access easily.
                landmark_name = lndmk.description.lower()
                landmark_info[landmark_name] = {"latitude":lndmk.locations[0].lat_lng.latitude, "longitude":lndmk.locations[0].lat_lng.longitude}
                
                
                lndmks = clean_text(lndmk.description)
                # we add the landmarks and image label to the dictionary to perform search
                for land_mark in lndmks:
                    if land_mark not in tagged_images:
                        tagged_images[land_mark] = []
                    if img_lbl not in tagged_images[land_mark]:
                        tagged_images[land_mark].append(img_lbl)
                
                # storing the landmarks with bounding boxes 
                lndmk_desc = '_'.join(lndmks)
                '_'.join(lndmks)
                if lndmk_desc not in obj_boxes:
                    obj_boxes[lndmk_desc] = list()
                ulx, uly, box_width, box_height = None, None, None, None
                ulx, uly = lndmk.bounding_poly.vertices[0].x, lndmk.bounding_poly.vertices[0].y
                box_width = lndmk.bounding_poly.vertices[1].x - lndmk.bounding_poly.vertices[0].x
                box_height = lndmk.bounding_poly.vertices[3].y - lndmk.bounding_poly.vertices[0].y
                if (ulx and uly and box_width and box_height) is not None:
                    vert = [ulx, uly, box_width, box_height] 
                    obj_boxes[lndmk_desc].append(vert)    

            for lgo in response.logo_annotations:
                
                logos = clean_text(lgo.description)
                # we add the logo names and image label to the dictionary to perform search
                for lgo_name in logos:
                    if lgo_name not in tagged_images:
                        tagged_images[lgo_name] = []
                    if img_lbl not in tagged_images[lgo_name]:
                        tagged_images[lgo_name].append(img_lbl)
                        
                lgo_desc = '_'.join(logos)
                if lgo_desc not in obj_boxes:
                    obj_boxes[lgo_desc] = list()
                ulx, uly, box_width, box_height = None, None, None, None
                ulx, uly = lgo.bounding_poly.vertices[0].x, lgo.bounding_poly.vertices[0].y
                box_width = abs(lgo.bounding_poly.vertices[1].x - lgo.bounding_poly.vertices[0].x)
                box_height = abs(lgo.bounding_poly.vertices[3].y - lgo.bounding_poly.vertices[0].y)
                if (ulx and uly and box_width and box_height) is not None:
                    vert = [ulx, uly, box_width, box_height]
                    obj_boxes[lgo_desc].append(vert)

            if len(response.localized_object_annotations) > 0:
                img_width, img_height = img.iiif["width"], img.iiif["height"]
            for lobj in response.localized_object_annotations:
                
                objects = clean_text(lobj.name)  
                # we add the object names and image label to the dictionary to perform search
                for obj_name in objects:
                    if obj_name not in tagged_images:
                        tagged_images[obj_name] = []
                    if img_lbl not in tagged_images[obj_name]:
                        tagged_images[obj_name].append(img_lbl)
                
                lobj_name = '_'.join(objects)
                if lobj_name not in obj_boxes:
                    obj_boxes[lobj_name] = list()
                ulx, uly, box_width, box_height = None, None, None, None
                ulx, uly = lobj.bounding_poly.normalized_vertices[0].x * img_width, lobj.bounding_poly.normalized_vertices[0].y * img_height
                box_width = (lobj.bounding_poly.normalized_vertices[1].x - lobj.bounding_poly.normalized_vertices[0].x) * img_width
                box_height = (lobj.bounding_poly.normalized_vertices[3].y - lobj.bounding_poly.normalized_vertices[0].y) * img_height
                if (ulx and uly and box_width and box_height) is not None:
                    vert = [ulx, uly, box_width, box_height]
                    obj_boxes[lobj_name].append(vert)

            for txt in response.text_annotations:
                modified_text = txt.description.replace(".", "_").lower()
                # we add the text and image label to the dictionary to perform search
                if modified_text not in tagged_images:
                    tagged_images[modified_text] = []
                if img_lbl not in tagged_images[modified_text]:
                        tagged_images[modified_text].append(img_lbl)    
                
                # the text identified on the images in not cleaned to store the original information.
                if modified_text not in obj_boxes:
                    obj_boxes[modified_text] = list()
                ulx, uly, box_width, box_height = None, None, None, None
                ulx, uly = txt.bounding_poly.vertices[0].x, txt.bounding_poly.vertices[0].y
                box_width = abs(txt.bounding_poly.vertices[1].x - txt.bounding_poly.vertices[0].x)
                box_height = abs(txt.bounding_poly.vertices[3].y - txt.bounding_poly.vertices[0].y)
                if (ulx and uly and box_width and box_height) is not None:
                    vert = [ulx, uly, box_width, box_height]
                    obj_boxes[modified_text].append(vert)

            # store the generated object boxes into the dictionary.
            annotated_images[img_lbl]["obj_boxes"] = obj_boxes
            
            # store the generated land mark information into the dictionary.
            if landmark_info != dict():
                annotated_images[img_lbl]["landmark_info"] = landmark_info
            
            # store the generated land mark information into the dictionary.
            annotated_images[img_lbl]["country"] = img_country
            
            # get the feature vector of the image
            try:
                feature_vec = get_vector(img.get_photo_link())
                image_vecs[img_lbl] = feature_vec.tolist()
            except:
                fvector_failed_images.apped(img_lbl)

100%|██████████| 40/40 [04:37<00:00,  6.94s/it]


## Saving the dictionaries to JSON file

In [10]:
tagged_images

{'monochrom': ['F25_125_recto',
  'T41_48_recto',
  'Cina5_350_recto',
  'T62_10_recto',
  'Cina5_254_recto',
  'F19_157_recto',
  'T4_9_recto',
  'T41_135_recto',
  'T31_6_recto',
  'T63_45_recto',
  'T5_4_recto',
  'T12_144_recto',
  'F25_121_recto',
  'F19_27_recto',
  'Cina5_331_recto',
  'T35_220_recto',
  'T27_3_recto',
  'T36_34_recto',
  'F26_117_recto',
  'F56_17_recto',
  'T10_104_recto',
  'F19_189_recto',
  'T11_180_recto',
  'T10_96_recto',
  'Cina4_34_recto',
  'T27_16_recto',
  'F19_62_recto',
  'T5_56_recto'],
 'snapshot': ['F25_125_recto',
  'T41_48_recto',
  'Cina5_350_recto',
  'T62_10_recto',
  'Cina5_254_recto',
  'T60_162_recto',
  'F19_157_recto',
  'T4_9_recto',
  'T41_135_recto',
  'T41_107_recto',
  'T31_6_recto',
  'T63_45_recto',
  'T5_4_recto',
  'T12_144_recto',
  'F25_121_recto',
  'F19_27_recto',
  'Cina5_331_recto',
  'T49_195_recto',
  'T35_220_recto',
  'T27_3_recto',
  'T36_34_recto',
  'F26_117_recto',
  'T31_30_recto',
  'T10_104_recto',
  'T49_242

In [11]:
annotated_images

{'F25_125_recto': {'iiif': {'@id': 'http://dl.cini.it/oa/items/65280/canvas.json',
   'label': 'F25_125_recto',
   '@type': 'sc:Canvas',
   'width': 3859,
   'height': 3133,
   'images': [{'@id': 'http://dl.cini.it/oa/files/64240/anno.json',
     'motivation': 'sc:painting',
     '@type': 'oa:Annotation',
     'resource': {'@id': 'http://dl.cini.it/files/original/78d54f4f33ded74c3c5087fe5aed42f4.jpg',
      '@type': 'dctypes:Image',
      'format': 'image/jpeg',
      'width': 3859,
      'height': 3133,
      'service': {'@id': 'http://dl.cini.it:8080/digilib/Scaler/IIIF/78d54f4f33ded74c3c5087fe5aed42f4.jpg',
       '@context': 'http://iiif.io/api/image/2/context.json',
       'profile': 'http://iiif.io/api/image/2/level2.json'}},
     'on': 'http://dl.cini.it/oa/items/65280/canvas.json'}],
   'metadata': [{'label': 'Identifier',
     'value': 'e2150d31-54c4-4a4c-8981-1d60be8f113b'},
    {'label': 'notes', 'value': 'F25_125_recto'},
    {'label': 'originalFileName', 'value': 'F25_125_

In [16]:
image_vecs

{'F25_125_recto': [2.1383938789367676,
  3.015453815460205,
  1.1989243030548096,
  1.18015456199646,
  0.11081298440694809,
  0.8492535948753357,
  2.16157865524292,
  1.2935435771942139,
  0.027188656851649284,
  1.4429482221603394,
  0.5057110786437988,
  0.7860021591186523,
  1.832258939743042,
  0.012645994313061237,
  0.2068813443183899,
  0.31207385659217834,
  0.476804256439209,
  3.0102317333221436,
  0.17640580236911774,
  0.20774449408054352,
  1.5987788438796997,
  2.4286985397338867,
  0.13517820835113525,
  1.4835489988327026,
  0.029670238494873047,
  0.59844571352005,
  0.6249019503593445,
  0.0,
  0.4794656038284302,
  1.534476637840271,
  2.3796632289886475,
  0.5731493234634399,
  0.6705460548400879,
  0.7989939451217651,
  1.7536574602127075,
  0.010188941843807697,
  5.028312683105469,
  1.1906774044036865,
  1.9446367025375366,
  1.2677791118621826,
  0.0793306976556778,
  0.32453951239585876,
  1.6906213760375977,
  1.079420804977417,
  0.46030595898628235,
  0.4

In [18]:
with open('tagged_images.json', 'w') as fp:
    json.dump(tagged_images, fp, indent=4)
    
with open('annotated_images.json', 'w') as fp:
    json.dump(annotated_images, fp, indent=4)

if len(annotation_failed_images) > 0:
    print("There are failed images")
    with open('annotation_failed_images.json', 'w') as fp:
        json.dump(annotation_failed_images, fp, indent=4)

with open('image_vectors.json', 'w') as fp:
    json.dump(image_vecs, fp, indent=4)
    
if len(fvector_failed_images) > 0:
    print("There are failed images")
    with open('fvector_failed_images.json', 'w') as fp:
        json.dump(fvector_failed_images, fp, indent=4)

## Inserting the data into Mangodb

In [19]:
# creating a client to work with mango db
mangoclient = pymongo.MongoClient(MANGO_CLIENT_URI)
# selecting the <terzani_photos> database
mango_db = mangoclient["terzani_photos"]

### Storing the Image Tags

In [20]:
# creating a new collection named <sample_tagging>
mango_tag_collection = mango_db["sample_taggings"]
# inserting the dictionary into the db
for tag, img_labels in tagged_images.items():
    mango_tag_collection.insert_one({"tag":tag,"images":img_labels})

### Storing the Image information

In [21]:
# creating a new collection named <sample_annotations>
mango_box_collection = mango_db["sample_annotations"]
# inserting the dictionary into the db
for img_label, annotation in annotated_images.items():
    mango_box_collection.insert_one({"label":img_label,"annotation":annotation})

### Storing the Image feature vector

In [22]:
# creating a new collection named <sample_tagging>
mango_tag_collection = mango_db["sample_image_vecs"]
# inserting the dictionary into the db
for img_label, img_vec in image_vecs.items():
    mango_tag_collection.insert_one({"image":img_label,"feature_vec":img_vec})